In [2]:
import sys
import os

# Define the relative path to the package
relative_path = '../Scrapping/Testing_Files'

# Convert the relative path to an absolute path
package_path = os.path.abspath(relative_path)

# Add the package path to the system path
sys.path.append(package_path)

# Now you can import your package
import DB_and_Azure

In [ ]:
import requests
def get_image_content(image_url):
    response = requests.get(image_url)
    return response.content